# core

> Fill in a module description here

In [1]:
#| default_exp

In [2]:
#| hide
from nbdev.showdoc import *

import os

from google.oauth2.credentials import Credentials
from google.auth.transport.requests import Request
from google_auth_oauthlib.flow import InstalledAppFlow

import google

In [3]:
#| export
class GetCreds_AuthException(Exception):
    def __init__(self):
        super().__init__( "must pass `token_file_path` or `creds` of type google.oauth2.credentials.Credentials class to get_creds()")

class GetCreds_InvalidTokenPath(Exception):
    def __init__(self):
        super().__init__( "invalid `token_file_path`.  Correct or pass `creds` of type google.oauth2.credentials.Credentials class to get_creds()")


In [36]:
# | export

class GenerateTokenFile_RequiresClientInfo(Exception):
    def __init__(self):
        super().__init__("must pass client_secret_path or client_secret_info, this information can be downloaded from https://console.cloud.google.com/apis/credentials/oauthclient")


def generate_token_file(token_file_path: str = 'env_token.json',
                        client_secret_path: str = None,
                        client_secret_info: dict = None,
                        scopes: [str] = ['https://www.googleapis.com/auth/drive.metadata.readonly',
                                         'https://www.googleapis.com/auth/drive.readonly']
                        ):

    if not client_secret_info and not client_secret_path:
        raise GenerateTokenFile_RequiresClientInfo()

    if client_secret_path:
        flow = InstalledAppFlow.from_client_secrets_file(client_secret_path, scopes)
    
    elif client_secret_info:
        flow = InstalledAppFlow.from_client_config(client_secret_info, scopes)
    
    creds = flow.run_local_server(port= 0,)

    # Save the credentials for the next run
    with open(token_file_path, 'w') as token:
        token.write(creds.to_json())


client_secret_info=json.loads(os.environ['GDOC_KEY'])

client_secret_info['installed']['redirect_uris'].append(
    # 'https://*.github.dev/extension-auth-callback', 
    'https://organic-space-guacamole-p55wrp49rwh7667.github.dev')
print(client_secret_info)

generate_token_file(client_secret_info = client_secret_info)



{'installed': {'client_id': '946705609741-a6fp61k4rgbcd1j5j2km9l1lbm82512h.apps.googleusercontent.com', 'project_id': 'iconic-woods-248903', 'auth_uri': 'https://accounts.google.com/o/oauth2/auth', 'token_uri': 'https://oauth2.googleapis.com/token', 'auth_provider_x509_cert_url': 'https://www.googleapis.com/oauth2/v1/certs', 'client_secret': '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', 'redirect_uris': ['http://localhost', 'https://organic-space-guacamole-p55wrp49rwh7667.github.dev']}}
Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=946705609741-a6fp61k4rgbcd1j5j2km9l1lbm82512h.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A34563%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.metadata.readonly+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.readonly&state=FEtdnswQQpkB8FDZcuuok8Ugz0tquZ&access_type=offline


In [34]:
# os.environ['HOSTNAME']

from pprint import pprint

pprint(os.environ.__dict__)

{'_data': {b'BROWSER': b'/vscode/bin/linux-x64/6c3e3dba23e8fadc360aed75ce363b'
                       b'a185c49794/bin/helpers/browser.sh',
           b'CLICOLOR': b'1',
           b'CLICOLOR_FORCE': b'1',
           b'CLOUDENV_ENVIRONMENT_ID': b'63262086-7ad4-4e44-8b2b-91d57a2dfba5',
           b'CODESPACES': b'true',
           b'CODESPACE_NAME': b'organic-space-guacamole-p55wrp49rwh7667',
           b'CONDA_DIR': b'/opt/conda',
           b'CONDA_SCRIPT': b'/opt/conda/etc/profile.d/conda.sh',
           b'ContainerVersion': b'13',
           b'DEBIAN_FLAVOR': b'focal-scm',
           b'DOCKER_BUILDKIT': b'1',
           b'DOJO_CREDS': b'{"domo_username" : "jae@onyxreporting.com",\n"dom'
                          b'o_password": "03u64goi8SX;M#I2zzId"\n}',
           b'DOJO_PASSWORD': b'03u64goi8SX;M#I2zzId',
           b'DOMO_DOJO_ACCESS_TOKEN': b'2f4cb212853732ef796abd46077cf1162c0a'
                                      b'3ed7cf1ff079',
           b'DOTNET_ROOT': b'/usr/local/dotne

#### sample implementation of get_creds

In [ ]:
def get_creds(token_file_path: str = None,
              credentials_file_pathcredentials_file_path: str = None,
              creds: google.oauth2.credentials.Credentials = None):
    """generate project and associated creds for google drive project here, https://console.cloud.google.com/apis/credentials/oauthclient"""

    if not creds and not token_file_path:
        raise GetCreds_AuthException()

    if not creds and not os.path.exists(token_file_path):
        raise GetCreds_InvalidTokenPath()

    # If modifying these scopes, delete the file token.json.

    # The file token.json stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.

    if not creds:
        creds = Credentials.from_authorized_user_file(token_file_path, SCOPES)

    # If there are no (valid) credentials available, let the user log in.
    if not creds.valid:
        if creds.expired and creds.refresh_token:
            creds.refresh(Request())

        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                credentials_file_path, SCOPES)
            creds = flow.run_local_server(port=0)

        # Save the credentials for the next run
        with open(token_file_path, 'w') as token:
            token.write(creds.to_json())

    return creds


In [14]:
import json
import os
from google.oauth2.credentials import Credentials

user_info = json.loads(os.environ['GDOC_KEY'])
creds = Credentials.from_authorized_user_info( user_info['installed'])
                                              

# get_creds(creds = creds)

# creds.valid
# creds.__dict__

ValueError: Authorized user info was not in the expected format, missing fields refresh_token.

In [6]:
#| hide
import nbdev; nbdev.nbdev_export()

TypeError: ExportModuleProc._default_exp_() missing 1 required positional argument: 'exp_to'